In [1]:
# Cell 1: Setup and Parse Stock List
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from script import parse_stocks_file

ROOT = Path.cwd()
stocks_file = ROOT / 'stocks.txt'

# Parse stocks.txt to get individual tickers and baskets
individual_tickers, baskets = parse_stocks_file(stocks_file)

print(f"Individual tickers: {individual_tickers}")
print(f"\nBaskets: {baskets}")

Individual tickers: ['TSLA', 'NVDA', 'MSFT', 'META', 'PLTR', 'MSTR', 'ASML', 'AMD', 'AVGO', 'ALAB', 'MRVL', 'BTC-USD', 'SOL-USD']

Baskets: {'Main AI Basket': ['TSLA', 'NVDA', 'MSFT', 'META', 'PLTR', 'ASML'], 'Lower Risk AI Basket': ['AMD', 'AVGO', 'ALAB', 'MRVL']}


In [2]:
# Cell 2: Run Batch Analysis
from script import analyze_ticker, analyze_basket

def run_batch(individual_tickers, baskets, concurrency=6, daily_bars=60, weekly_bars=52):
    """Run analysis on all individual tickers and baskets"""
    results = []
    
    # Analyze individual tickers in parallel
    with ThreadPoolExecutor(max_workers=concurrency) as ex:
        futures = {ex.submit(analyze_ticker, t, daily_bars, weekly_bars): t for t in individual_tickers}
        for fut in as_completed(futures):
            results.append(fut.result())
    
    # Analyze baskets (market cap weighted aggregations)
    for basket_name, constituents in baskets.items():
        basket_result = analyze_basket(basket_name, constituents, daily_bars, weekly_bars)
        results.append(basket_result)
    
    return pd.DataFrame(results)

# Run the analysis
df = run_batch(individual_tickers, baskets)
df = df.round(2)

# Print Summary Stats
print(f"\nAnalysis complete: {len(df)} rows ({len(individual_tickers)} tickers + {len(baskets)} baskets)")
print("FULL HOLD + ADD tickers:", ', '.join(df[df['signal'] == 'FULL HOLD + ADD']['ticker']))

df


Analysis complete: 15 rows (13 tickers + 2 baskets)
FULL HOLD + ADD tickers: PLTR, TSLA, NVDA, ASML, MRVL, [Main AI Basket]


,ticker,signal,current_price,price_note,date,d20,d50,d100,d200,w10,...,weekly_val,s1,s2,s3,r1,r2,r3,notes,confluence,recommendation
0,PLTR,FULL HOLD + ADD,167.86,last close,2026-01-04,184.55,180.94,176.47,151.04,179.10,...,69.16,147.56,142.34,128.51,207.52,190.00,188.20,,Caution / Skip,No Buy – Wait for Better Confluence
1,META,HOLD MOST → REDUCE,650.41,last close,2026-01-04,658.59,651.82,698.45,672.61,640.47,...,586.28,580.78,578.18,546.88,795.06,789.62,758.54,,Neutral / Acceptable,No Buy
2,TSLA,FULL HOLD + ADD,438.07,last close,2026-01-04,464.27,445.01,418.45,360.20,442.77,...,219.94,411.45,382.78,373.04,488.54,474.07,470.75,,Neutral / Acceptable,Immediate Starter Possible on Minor Dip
3,MSFT,HOLD MOST → REDUCE,472.94,last close,2026-01-04,483.17,495.21,503.04,476.73,485.44,...,410.33,464.89,407.71,404.37,553.50,552.69,530.04,,Neutral / Acceptable,No Buy
4,NVDA,FULL HOLD + ADD,188.85,last close,2026-01-04,182.96,186.50,183.02,160.48,183.82,...,86.60,176.75,169.54,164.05,212.18,NaN,NaN,,Bullish / Favor Add,No Starter – Wait for Primary Dip
5,MSTR,CASH,157.16,last close,2026-01-04,167.75,202.40,267.75,320.20,179.39,...,235.84,155.61,120.23,113.69,543.00,457.22,430.35,,Neutral / Acceptable,No Buy
6,AMD,HOLD,223.47,last close,2026-01-04,214.41,227.63,203.23,163.23,219.71,...,129.85,194.28,161.81,153.34,267.08,227.30,NaN,,Neutral / Acceptable,No Buy
7,ASML,FULL HOLD + ADD,1163.78,last close,2026-01-04,1083.21,1056.14,965.47,843.33,1063.95,...,574.25,988.40,946.11,933.75,NaN,NaN,NaN,,Caution / Skip,No Buy – Wait for Better Confluence
8,ALAB,HOLD MOST → REDUCE,179.56,last close,2026-01-04,162.40,160.76,179.74,134.57,161.72,...,107.54,148.51,131.42,84.78,262.90,201.86,199.47,,Caution / Skip,No Buy
9,MRVL,FULL HOLD + ADD,89.39,last close,2026-01-04,87.76,87.60,82.36,74.35,87.82,...,58.19,79.06,73.62,69.71,127.15,125.76,102.77,,Bullish / Favor Add,No Starter – Wait for Primary Dip


In [3]:
# Cell 3: Print Buy Summary
def print_buy_summary(df, cash_available=118305):
    print("\n### Larsson Portfolio Buy Summary – Next Trading Day\n")
    print("**Rule-Based Only** — Conservative phased entry rules locked in:")
    print("- No immediate starters in Bullish zones (wait for better prices).")
    print("- Primary adds only on real weakness to predefined green supports.\n")
    
    print(f"**Cash Available**: ~${cash_available:,.0f} (~59% dry powder)\n")
    
    # Load target allocations from targets.csv
    targets_file = ROOT / 'targets.csv'
    if targets_file.exists():
        targets_df = pd.read_csv(targets_file)
        target_dict = dict(zip(targets_df['ticker'], targets_df['target_pct']))
        value_dict = dict(zip(targets_df['ticker'], targets_df['target_value']))
    else:
        target_dict = {}
        value_dict = {}
        print("⚠️  targets.csv not found - using N/A for target percentages\n")
    
    # Use exact match instead of contains
    eligible = df[df['signal'] == "FULL HOLD + ADD"]
    if eligible.empty:
        print("No FULL HOLD + ADD names — no buys recommended.")
        return
    
    def get_primary_zone(row):
        """Hybrid conservative primary add zone: Lower Value Area + Key Long-Term SMAs"""
        val = row['daily_val']
        poc = row['daily_poc']
        d100 = row['d100']
        d200 = row['d200']
        
        # Handle NaN/missing
        if pd.isna(val) or pd.isna(poc):
            return f"Near Key SMAs (D100 ${int(d100)} / D200 ${int(d200)})"
        
        lower_va = f"Lower Value Area (${int(val)}–${int(poc)})"
        sma_part = f"or Key Long-Term SMA (D100 ${int(d100)} / D200 ${int(d200)})"
        return f"{lower_va} {sma_part}"
    
    print("| Ticker | Target % | Current Price (Close) | Confluence | Buy Recommendation | Primary Add (40–50% of target) | Primary Add Zone (Conservative) | Approx Shares at Zone |")
    print("|--------|----------|-----------------------|------------|------------------------------|--------------------------------|---------------------------------|-----------------------|")
    
    for _, row in eligible.iterrows():
        ticker = row['ticker']
        price = row['current_price']
        confluence = row['confluence']
        rec = row['recommendation']
        
        # Get target % and value from config file
        target_pct = target_dict.get(ticker, 'N/A')
        target_val = value_dict.get(ticker, 4001)
        
        if target_pct != 'N/A':
            target_pct_str = f"{target_pct}%"
        else:
            target_pct_str = 'N/A'
        
        # Calculate primary add amounts (40-50% of target)
        primary_low = target_val * 0.4
        primary_high = target_val * 0.5
        
        # Get primary zone
        zone = get_primary_zone(row)
        
        # Share estimate using approximate zone midpoint (or current price fallback)
        zone_mid_est = price * 0.9  # rough 10% dip estimate for conservatism
        shares_low = int(primary_low / zone_mid_est)
        shares_high = int(primary_high / zone_mid_est)
        shares_str = f"{shares_low}–{shares_high} shares"
        
        print(f"| **{ticker}** | {target_pct_str} | ${price:.2f} | **{confluence}** | {rec} | ~${primary_low:,.0f}–${primary_high:,.0f} | {zone} | {shares_str} |")
    
    print("\n**No Buy Action**")
    print("- All other names: Not FULL HOLD + ADD or confluence insufficient.\n")
    print("**Execution Plan**")
    print("- No limits to place / Wait for weakness to primary zones / etc.")

print_buy_summary(df)



### Larsson Portfolio Buy Summary – Next Trading Day

**Rule-Based Only** — Conservative phased entry rules locked in:
- No immediate starters in Bullish zones (wait for better prices).
- Primary adds only on real weakness to predefined green supports.

**Cash Available**: ~$118,305 (~59% dry powder)

| Ticker | Target % | Current Price (Close) | Confluence | Buy Recommendation | Primary Add (40–50% of target) | Primary Add Zone (Conservative) | Approx Shares at Zone |
|--------|----------|-----------------------|------------|------------------------------|--------------------------------|---------------------------------|-----------------------|
| **PLTR** | 5% | $167.86 | **Caution / Skip** | No Buy – Wait for Better Confluence | ~$4,000–$5,000 | Lower Value Area ($169–$181) or Key Long-Term SMA (D100 $176 / D200 $151) | 26–33 shares |
| **TSLA** | 50% | $438.07 | **Neutral / Acceptable** | Immediate Starter Possible on Minor Dip | ~$40,006–$50,007 | Lower Value Area ($403–$439) or 

In [4]:
# Cell 4: Export Results
import shutil
from datetime import datetime

# Add timestamp to avoid overwriting
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_file = ROOT / f'batch_results_{timestamp}.csv'

# Save to workspace
df.to_csv(output_file, index=False)
print(f"✅ Results exported to: {output_file}")

# Copy to Downloads folder
downloads_path = Path.home() / 'Downloads' / f'batch_results_{timestamp}.csv'
try:
    shutil.copy(output_file, downloads_path)
    print(f"✅ File also copied to: {downloads_path}")
except PermissionError:
    print(f"⚠️  Could not copy to Downloads - file may be open in another program")
    print(f"   Close the file and try again, or use: {output_file}")

✅ Results exported to: C:\workspace\my_script_project\batch_results_20260104_1709.csv
✅ File also copied to: C:\Users\karms\Downloads\batch_results_20260104_1709.csv


In [5]:
# Cell 5: Cleanup Old Results
from pathlib import Path

def cleanup_old_results(keep_latest=1):
    """Keep only the most recent batch result file"""
    batch_files = sorted(
        ROOT.glob('batch_results_*.csv'),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    
    for old_file in batch_files[keep_latest:]:
        print(f"🗑️  Deleting: {old_file.name}")
        old_file.unlink()
    
    print(f"✅ Kept {min(len(batch_files), keep_latest)} most recent file(s)")

# Auto-cleanup after each run:
cleanup_old_results(keep_latest=1)

🗑️  Deleting: batch_results_20260104_1638.csv
✅ Kept 1 most recent file(s)
